
# Einstein Wave Propagation

## The Einstein's wave Equation

$$
\begin{cases}
\partial^2 _{tt} S \gamma  = -\text{inc }\gamma\\
\nabla \cdot  S\gamma  =0\\
\nabla \cdot \nabla \cdot  S\gamma  =0\\
\end{cases}
$$

Using the change of variabl $S\gamma = \kappa$ and with some observations (i.e. the second constrint equation is solved by the firs one) we have that :

$$
\begin{cases}
\varepsilon \kappa_t  = -\text{inc}\gamma\\
d_t = - \text{curl curl }e \\
\text{ div } d = 0
\end{cases}
$$









In [2]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
import matplotlib.pyplot as plt
import time

import scipy.sparse as sp
from scipy.sparse import csc_matrix
import matplotlib.pylab as plt

In [3]:
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)

def Div(u):
    if u.dim == 3:
        return CF( (u[0].Diff(x)+ u[1].Diff(y) + u[2].Diff(z)) )
    if u.dim == 9:
        return CF( (Div(u[0,:]),Div(u[1,:]),Div(u[2,:])) )

def tr(u):
    return (u[0,0]+u[1,1]+u[2,2])

def IncOperator():
    pass

In [4]:
def EinsteinPropagation(mesh , order = 1 , initail_g_field = None , bonus_intorder = 10 ,draw_g_field = True , draw_k_field = False, draw_divk_field = False , \
                    final_time  = 0.1, step_time = 0.002 , MoreInfo = False,video = False):
    
    n = specialcf.normal(3)
    P_n = OuterProduct(n,n) 
    Q_n = Id(3) - OuterProduct(n,n) 
    
    ######################################
    #     FES                    #
    ######################################
    if MoreInfo:
        start = time.time()
        print("create the spaces :")

    fes =  HCurlCurl(mesh, order=order )*HDivDiv(mesh, order=order   , dirichlet= ".*")
    fes_projection =  HDivDiv(mesh, order=order   , dirichlet= ".*")*HCurl(mesh,  order=order-1)

    if MoreInfo:
        end = time.time()
        print(str(end-start)+"\n")

    ######################################
    #            electric field          #
    ######################################
    
    if MoreInfo:
        start = time.time()
        print("interpolate the initaial weak gravitational field field :")

    weak_field = GridFunction(fes) 
    gf_g, gf_k = weak_field.components
    gf_k.Set ( initail_g_field, bonus_intorder=bonus_intorder)#, dual=True)

    if MoreInfo:
        end = time.time()
        print(str(end-start)+"\n")

    # projectional part
    projection_field = GridFunction(fes_projection) 
    Pk, Pq = projection_field.components

    # trial and test function for H(curl) X H(div)
    uc, ud , = fes.TrialFunction()
    vc, vd , = fes.TestFunction()

    # create the inverse of the submatrix M_e 
    if MoreInfo:
        start = time.time()
        print("create the inverse of the submatrix M_g  :")

    ug,vg = HCurlCurl(mesh, order=order ).TnT()
    gdofs = fes.Range(0)
    emb_g = Embedding(fes.ndof, gdofs)
    inv_g = BilinearForm(InnerProduct(ug,vg)*dx).Assemble().mat.Inverse(inverse="sparsecholesky")
    
    if MoreInfo:
        end = time.time()
        #plt.spy(inv_g)
        print(str(end-start)+"\n")


    #create the inverse of M_k 
    if MoreInfo:
        start = time.time()
        print("create the inverse of the submatrix M_k  :")
    uk,vk = HDivDiv(mesh, order=order   , dirichlet= ".*").TnT()
    kdofs = fes.Range(1)
    emb_k = Embedding(fes.ndof, kdofs)
    inv_k = BilinearForm(InnerProduct(uk,vk)*dx).Assemble().mat.Inverse(inverse="sparsecholesky")
    
    if MoreInfo:
        end = time.time()
        #plt.spy(csc_matrix(inv_k.CSR()))
        print(str(end-start)+"\n")

    # create the passage matrix M_de : H(curl) -> H(div)
    if MoreInfo:
        start = time.time()
        print("create the passage matrix M_de : H(curl) -> H(div)  :")

    u = HCurlCurl(mesh, order=order ).TrialFunction()
    v = HDivDiv(mesh, order=order   , dirichlet= ".*").TestFunction()
    M_kg = BilinearForm(trialspace=HCurlCurl(mesh, order=order ), testspace=HDivDiv(mesh, order=order   , dirichlet= ".*") , geom_free = True)
    M_kg +=  InnerProduct(u, v)*dx
    M_kg.Assemble()

    if MoreInfo:
        end = time.time()
        #plt.spy(csc_matrix(M_kg.mat.CSR()))
        print(str(end-start)+"\n")


    # create the passage matrix M_ed : H(curl) -> H(div)
    if MoreInfo:
        start = time.time()
        print("create the passage matrix M_ed : H(curl) -> H(div)  :")

    u = HCurlCurl(mesh, order=order ).TestFunction()
    v = HDivDiv(mesh, order=order   , dirichlet= ".*").TrialFunction()
    M_gk = BilinearForm(testspace=HCurlCurl(mesh, order=order ), trialspace=HDivDiv(mesh, order=order   , dirichlet= ".*"), geom_free = True)
    M_gk +=  InnerProduct(u, v)*dx
    M_gk.Assemble()

    if MoreInfo:
        end = time.time()
        #plt.spy(csc_matrix(M_gk.mat.CSR()))
        print(str(end-start)+"\n")

    # creating the matrix for the operator curl-curl:
    if MoreInfo:
        start = time.time()
        print("create the passage matrix for the operator curl-curl  :")

    u = HCurlCurl(mesh, order=order ).TestFunction()
    v = HCurlCurl(mesh, order=order ).TrialFunction()
    print("create geometric elements such as norm, tangents ...")
    n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
    t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
    t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
    e = specialcf.tangential(3,True)
    n1 = Cross( t1, e)
    n2 = Cross( t2, e) 

    print("create the discrete-distributional inc operator")

    K = BilinearForm(HCurlCurl(mesh, order=order ))
    K += -InnerProduct(u.Operator("inc"),v)*dx  
    K += -( InnerProduct(Q_n*n_cross_v, curl(u).trans) + Cross(u*n,n)*(curl(v)*n) )*dx(element_vb=BND)
    K += -(u[n1,e]*v[e,t1] - u[n2,e]*v[e,t2])*dx(element_vb=BBND)
    K.Assemble()
    
    print(" discrete-distributional inc operator created")

    if MoreInfo:
        end = time.time()
        print(str(end-start)+"\n")

    # We need to create the Operatro for S^-1:
    
    if MoreInfo:
        start = time.time()
        print("create the passage matrix for the operator J = S^{-1}  :")

    J = BilinearForm(HDivDiv(mesh, order=order ))
    J += InnerProduct(u,v)*dx  
    J += -InnerProduct(tr(u),tr(v))/2*dx
    J.Assemble()

    if MoreInfo:
        end = time.time()
        #plt.spy(csc_matrix(inv_S.CSR()))
        print(str(end-start)+"\n")
    # compose the 2 final matrices
    if MoreInfo:
        start = time.time()
        print("create the passage final matrices Mup Mdown  :")

    Mup = emb_g @ inv_g @ M_gk.mat @ J.mat @ inv_k  @ emb_k.T

    Mdown = emb_k @ inv_k @ M_kg.mat @ inv_g  @ K.mat @ emb_g.T 
    #M =  Mdown
    #   
    if MoreInfo:
        end = time.time()
        print(str(end-start)+"\n")

    # Create the projection operator for the divergence free part
    if MoreInfo:
        start = time.time()
        print("create the projection operator for the divergence free part  :")
    u, p  = fes_projection.TrialFunction()
    v, q  = fes_projection.TestFunction()

    P = BilinearForm(fes_projection)
    P +=  InnerProduct(v,u) * dx
    P +=  div( v)*p * dx
    P +=  -InnerProduct(v*n - v[n,n]*n , p - (p*n)*n ) * dx(element_boundary = True)
    P +=  div( u)*q * dx
    P +=  -InnerProduct(u*n - u[n,n]*n , q - (q*n)*n ) * dx(element_boundary = True)
    P += -1e-9*p*q * dx

    F = BilinearForm(fes_projection)
    F +=  InnerProduct(v,u) * dx

    P.Assemble()
    F.Assemble()

    invP = P.mat.Inverse(inverse="sparsecholesky")

    if MoreInfo:
        #print("invP")
        #plt.spy(csc_matrix(invP.CSR()))
        print("F")
        #plt.spy(csc_matrix(F.mat.CSR()))
        end = time.time()
        print(str(end-start)+"\n")

    if draw_g_field :
        print("draw g")
        scene_g = Draw(gf_g, mesh,clipping=(0,0,1),draw_surf=False)
    if draw_divk_field :
        print("draw div k")
        scene_divd = Draw(div(gf_k), mesh,clipping=(0,0,1) ,draw_surf=False)
    if draw_k_field :
        print("draw k")
        scene_k = Draw(gf_k, mesh,clipping=(0,0,1))
    
    if MoreInfo:
        start = time.time()
    t =   final_time
    tau = step_time
    h = 0

    
    # Need to fix this part, gererates too many trace files
    #
    #if MoreInfo :
    #    TM = TaskManager(pajetrace=10**8)
    #else :
    #    TM =TaskManager()
    while h < t :
        input("sometuibh")
        h+=tau
        #we use only one matrix of the form described in the above markdowm
        #input("top part :")
        weak_field.vec.data += tau * Mdown *weak_field.vec
        #input("top part :")
        weak_field.vec.data += tau * Mup *weak_field.vec
        #input("bot part : ")

        #elfield.vec.data += tau* (inv @ Mm ) @ inv * elfield.vec
        #scene_d.Redraw()
        
        Pk.vec.data = gf_k.vec
        projection_field.vec.data =  invP*(F.mat*projection_field.vec)
        gf_k.vec.data = Pk.vec
        
        if draw_g_field :
            scene_g.Redraw()
        if draw_divk_field :
            scene_divd.Redraw()
        if draw_k_field :
            scene_k.Redraw()
        #input("time : " +str(h))


    if MoreInfo :
        end = time.time()
        print("Time of execution for "+str(int(final_time/step_time)) + "steps is "+str(end-start))
    return gf_g,gf_k

In [5]:
final_time = 0.05
step_time = 0.005
MoreInfo = True
video = False
draw_divk_field = True
draw_k_field = True

Maxh = 0.17
order = 3
mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))

# electric permettivity and magnetic permeability
eps = CF((1), dims=(1,1))
mu =  CF((1), dims=(1,1))

# define initial conditions for the wave
peak = IfPos((1-(x-0.5)**2 -(y-0.5)**2 -(z-0.5)**2)**2,exp(25)*exp(-25/(1-(x-0.5)**2 -(y-0.5)**2 -(z-0.5)**2)) , 1)
Draw(peak,mesh, clipping=(1,1,1,1))
PEAK = CF( (peak, 0, 0 ,-peak,0,0  , 0,0,0), dims=(3,3))
#Draw(Inc(PEAK)[1,2], mesh, clipping=(1,1,0))
initail_g_field = CF ( Inc(PEAK), dims=(3,3))



WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

In [6]:

with TaskManager():
    g,k = EinsteinPropagation(mesh , order = order ,initail_g_field = initail_g_field , final_time  = final_time, step_time = step_time,MoreInfo = MoreInfo, video =video , draw_divk_field = draw_divk_field , draw_k_field = draw_k_field)

create the spaces :
0.01156163215637207

interpolate the initaial weak gravitational field field :
11.1680428981781

create the inverse of the submatrix M_g  :
3.193502426147461

create the inverse of the submatrix M_k  :
1.7168357372283936

create the passage matrix M_de : H(curl) -> H(div)  :
0.016954660415649414

create the passage matrix M_ed : H(curl) -> H(div)  :
0.012965202331542969

create the passage matrix for the operator curl-curl  :
create geometric elements such as norm, tangents ...
create the discrete-distributional inc operator


: 

: 